In [1]:
!pip install azure-storage-blob

     ---------------------------------------- 0.0/387.8 kB ? eta -:--:--
     ----------------- -------------------- 174.1/387.8 kB 3.5 MB/s eta 0:00:01
     -------------------------------------  378.9/387.8 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 387.8/387.8 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/174.5 kB ? eta -:--:--
     -------------------------------------  174.1/174.5 kB 5.1 MB/s eta 0:00:01
     -------------------------------------- 174.5/174.5 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ------ --------------------------------- 0.4/2.5 MB 7.6 MB/s eta 0:00:01
     ----------- ---------------------------- 0.7/2.5 MB 7.3 MB/s eta 0:00:01
     ---------------- ----------------------- 1.0/2.5 MB 7.0 MB/s eta 0:00:01
     -------------------- ------------------- 1.3/2.5 MB 7.3 MB/s eta 0:00:01
     ------------------------- -------------- 1.5/2.5 MB 7.0 MB/s eta 0

In [2]:
from azure.storage.blob import BlobServiceClient,BlobClient
import io
import pandas as pd
connection_string='DefaultEndpointsProtocol=https;AccountName=relgendy;AccountKey=YTQ1cy1bOeU59C/5cp+Fjk+/DKHvr9LlimYMwuB+IU9yUtMzmvAISJy9SH4sRC9z87AyOM5ZTCyX+AStInnX4g==;EndpointSuffix=core.windows.net'
container_name='historicaldata'

blob_service_client= BlobServiceClient.from_connection_string(connection_string)
blob_client=blob_service_client.get_blob_client(container_name,'Datasets/HistoricalData_1679044439359.csv')

blob_data =blob_client.download_blob()
data = blob_data.content_as_text()
df=pd.read_csv(io.StringIO(data))
print(df.head())

         Date Close/Last    Volume      Open      High        Low
0  03/16/2023    $155.85  76254420   $152.16   $156.46    $151.64
1  03/15/2023    $152.99  77167870   $151.19  $153.245    $149.92
2  03/14/2023    $152.59  73695890   $151.28   $153.40    $150.10
3  03/13/2023    $150.47  84457120  $147.805   $153.14    $147.70
4  03/10/2023    $148.50  68572400   $150.21   $150.94  $147.6096


In [ ]:
## QUERY TO CREATE 10 CONNECTIONS TO THE AZURE SQL-DB

#connection queue
from queue import Queue
from threading import Lock
import pyodbc
from getpass import getpass
server = 'lvtestserver.database.windows.net'
database = 'lucasvdb'
username = 'lucasvitantonio'
password = getpass()    # creates an input when executing the code to type in the pass
driver= '{ODBC Driver 18 for SQL Server}'
connection_string= 'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password

max_con=10
connection_pool=Queue(max_con)
lock=Lock() ## lock ensures concurrency. each query is executed one time but one person.

# creates an connection instance with the string connection
def create_connection():
    return pyodbc.connect(connection_string, autocommit=True)

# define a function to get connection from queue. If queue is empty
# it will create a new connection
def get_Connection():
    with lock:
        if not connection_pool.empty():
            return connection_pool.get()
    return create_connection()

## releases the connection and puts it back into queue
def release_connection(connection):
    with lock:
        connection_pool.put(connection)

connection=get_Connection()
cursor=connection.cursor()
cursor.execute("select * from SalesLT.Address WHERE CountryRegion = 'Canada' and City = 'Toronto'")

for row in cursor:
    print(row)

release_connection(connection)
# connection.close()

In [ ]:
## PUSHING DATA INTO A TABLE INSIDE THE CLOUD
cnx = get_Connection()
cursor = cnx.cursor()

queryInsert = "INSERT INTO Persons VALUES ('2' ,'Lucas', 'V', '5dfd', '5dsfs')"
# create_new_table = cursor.execute(query)
insert_values = cursor.execute(queryInsert)

cnx.commit()
cnx.close()

In [ ]:
## PUSHING A CSV BLOB FILE INTO A CONTAINER IN AZURE
import pandas as pd
from azure.storage.blob import BlobServiceClient,BlobClient,ContainerClient

connection_string= "DefaultEndpointsProtocol=https;AccountName=storagelucasv;AccountKey=nfshKMTaX1gRybJy+o4SYnrrqJJYAm5ooBwUKaZi7cm0V8PG4w7pQrn40O2qlRGhbJiH2iahS6sm+AStAJFMTA==;EndpointSuffix=core.windows.net"


data  = {'name':['John','Jane','Bob'],'age':[30,25,40]}
df =pd.DataFrame(data)

csv_data=df.to_csv(index=False)

## creating a connection client
blob_service_client=BlobServiceClient.from_connection_string(connection_string)

container_name="testcontainer"
container_Client = blob_service_client.get_container_client(container_name)
blob_name='test.csv'
blob_client=container_Client.get_blob_client(blob_name)
blob_client.upload_blob(csv_data, overwrite=True)